# Creating pose matrices from Quaternions

Following the examples in [From_pose_to_transform](./From_pose_to_transform.ipynb) and [From_pose_to_transform_opencv_camra](./From_pose_to_transform_opencv_camra), let's analyse how to build 4x4 Pose matrices from quaternions.

Let's use the [Woodscape dataset](https://github.com/valeoai/WoodScape) by Valeo.

## Woodscape documentation
From the Woodscape dataset we can read the following information:

"The vehicle coordinate system, which follows the ISO 8855 convention, is anchored to the ground below the midpoint of the rear axle. The X axis points in driving direction, the Y axis points to the left side of the vehicle and the Z axis points up from the ground. 
The camera sensor's coordinate system is based on OpenCV. The X axis points to the right along the horizontal sensor axis, the Y axis points downwards along the vertical sensor axis and the Z axis points in viewing direction along the optical axis to maintain the right-handed system. 
The values of the translation are given in meters and the rotation is given as a quaternion. They describe the coordinate transformation from the camera coordinate system to the vehicle coordinate system. The rotation matrix can be obtained using e.g. `scipy.spatial.transform.Rotation.from_quat`."

NOTE: The quaternion provided by Woodscape then follows Scipy convention, so (x, y, z, w), instead of (w, x, y, z)

NOTE: Woodscape has its own Python routines. Let's not use them to double check the validity of the presented approach.

Let's read a sample image and its corresponding calibration information:

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import json
import cv2
from vcd import utils
from scipy.spatial.transform import Rotation as SciRot
from matplotlib import pyplot as plt

def plot_cs(pose_wrt_ref, name, L=1):
    # Explore the coordinate systems defined for this scene
    axis = np.array([[0, L, 0, 0, 0, 0],
            [0, 0, 0, L, 0, 0],
            [0, 0, 0, 0, 0, L],
            [1, 1, 1, 1, 1, 1]])  # matrix with several 4x1 points
    pose_wrt_ref = np.array(pose_wrt_ref).reshape(4, 4)
    axis_ref = pose_wrt_ref.dot(axis)
    origin = axis_ref[:, 0]
    x_axis_end = axis_ref[:, 1]
    y_axis_end = axis_ref[:, 3]
    z_axis_end = axis_ref[:, 5]
    ax.plot([origin[0], x_axis_end[0]], [origin[1], x_axis_end[1]], [origin[2], x_axis_end[2]], 'r-')
    ax.plot([origin[0], y_axis_end[0]], [origin[1], y_axis_end[1]], [origin[2], y_axis_end[2]], 'g-')
    ax.plot([origin[0], z_axis_end[0]], [origin[1], z_axis_end[1]], [origin[2], z_axis_end[2]], 'b-')

    ax.text(origin[0], origin[1], origin[2], r'{}'.format(name))
    ax.text(x_axis_end[0], x_axis_end[1], x_axis_end[2], 'X')
    ax.text(y_axis_end[0], y_axis_end[1], y_axis_end[2], 'Y')
    ax.text(z_axis_end[0], z_axis_end[1], z_axis_end[2], 'Z')

# load example image
fisheye_fv_image = cv2.imread('../data/Woodscape/00000_FV.png')
fisheye_rv_image = cv2.imread('../data/Woodscape/00086_RV.png')
fisheye_mvl_image = cv2.imread('../data/Woodscape/00141_MVL.png')
fisheye_mvr_image = cv2.imread('../data/Woodscape/00096_MVR.png')

# Read file: intrinsics, extrinsics
with open('../data/Woodscape/00000_FV.json') as f:
    config_fv = json.load(f)

intrinsic_fv = config_fv['intrinsic']
coefficients_fv = [intrinsic_fv['k1'], intrinsic_fv['k2'], intrinsic_fv['k3'], intrinsic_fv['k4']]
traslation_fv = config_fv['extrinsic']['translation']
principal_point_fv = (intrinsic_fv['cx_offset'], intrinsic_fv['cy_offset'])
size_fv = (intrinsic_fv['width'], intrinsic_fv['height'])
quaternion_fv = config_fv['extrinsic']['quaternion']

print(traslation_fv)
print(quaternion_fv)

In the Woodscape Python API, the rotation matrix is built using SciRot.

In [ ]:
rotation_fv = SciRot.from_quat(quaternion_fv).as_matrix()
print(rotation_fv)

def q2R(q):
    assert(q.shape == (4, 1))
    x = q[0, 0]
    y = q[1, 0]
    z = q[2, 0]
    w = q[3, 0]
    
    x2 = x*x
    y2 = y*y
    z2 = z*z
    xy = x*y
    xz = x*z
    yz = y*z
    xw = x*w
    yw = y*w
    zw = z*w

    R1 = np.array([[1-2*y2-2*z2, 2*xy-2*zw, 2*xz+2*yw],
                  [2*xy+2*zw, 1-2*x2-2*z2, 2*yz-2*xw],
                  [2*xz-2*yw, 2*yz+2*xw, 1-2*x2-2*y2]])
    return R1

rotation_fv = q2R(np.array(quaternion_fv).reshape(4,1))
print(rotation_fv)




Let's build the pose:

In [ ]:
R_fv_wrt_lcs = np.array(rotation_fv)
C_fv_wrt_lcs = np.transpose(np.array([traslation_fv]))
P_fv_wrt_lcs = utils.create_pose(R_fv_wrt_lcs, C_fv_wrt_lcs)

# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

Let's do the same for the other cameras:

In [ ]:
import timeit
# Let's measure computation of R from q
def scipy(q):
    return SciRot.from_quat(q).as_matrix()

qa = quaternion_fv
qb = np.array(quaternion_fv).reshape(4,1)
loop = 10000

result_scipy = timeit.timeit('scipy(qa)', globals=globals(), number=loop)
result_q2R = timeit.timeit('q2R(qb)', globals=globals(), number=loop)
print(result_scipy)
print(result_q2R)


In [ ]:
# Read file: intrinsics, extrinsics
with open('../data/Woodscape/00086_RV.json') as f:
    config_rv = json.load(f)
intrinsic_rv = config_rv['intrinsic']
coefficients_rv = [intrinsic_rv['k1'], intrinsic_rv['k2'], intrinsic_rv['k3'], intrinsic_rv['k4']]
traslation_rv = config_rv['extrinsic']['translation']
principal_point_rv = (intrinsic_rv['cx_offset'], intrinsic_rv['cy_offset'])
size_rv = (intrinsic_rv['width'], intrinsic_rv['height'])
quaternion_rv = config_rv['extrinsic']['quaternion']
rotation_rv = SciRot.from_quat(quaternion_rv).as_matrix()
R_rv_wrt_lcs = np.array(rotation_rv)
C_rv_wrt_lcs = np.transpose(np.array([traslation_rv]))
P_rv_wrt_lcs = utils.create_pose(R_rv_wrt_lcs, C_rv_wrt_lcs)

print(traslation_rv)
print(quaternion_rv)

with open('../data/Woodscape/00141_MVL.json') as f:
    config_mvl = json.load(f)
intrinsic_mvl = config_mvl['intrinsic']
coefficients_mvl = [intrinsic_mvl['k1'], intrinsic_mvl['k2'], intrinsic_mvl['k3'], intrinsic_mvl['k4']]
traslation_mvl = config_mvl['extrinsic']['translation']
principal_point_mvl = (intrinsic_mvl['cx_offset'], intrinsic_mvl['cy_offset'])
size_mvl = (intrinsic_mvl['width'], intrinsic_mvl['height'])
quaternion_mvl = config_mvl['extrinsic']['quaternion']
rotation_mvl = SciRot.from_quat(quaternion_mvl).as_matrix()
R_mvl_wrt_lcs = np.array(rotation_mvl)
C_mvl_wrt_lcs = np.transpose(np.array([traslation_mvl]))
P_mvl_wrt_lcs = utils.create_pose(R_mvl_wrt_lcs, C_mvl_wrt_lcs)

print(traslation_mvl)
print(quaternion_mvl)

with open('../data/Woodscape/00096_MVR.json') as f:
    config_mvr = json.load(f)
intrinsic_mvr = config_mvr['intrinsic']
coefficients_mvr = [intrinsic_mvr['k1'], intrinsic_mvr['k2'], intrinsic_mvr['k3'], intrinsic_mvr['k4']]
traslation_mvr = config_mvr['extrinsic']['translation']
principal_point_mvr = (intrinsic_mvr['cx_offset'], intrinsic_mvr['cy_offset'])
size_mvr = (intrinsic_mvr['width'], intrinsic_mvr['height'])
quaternion_mvr = config_mvr['extrinsic']['quaternion']
rotation_mvr = SciRot.from_quat(quaternion_mvr).as_matrix()
R_mvr_wrt_lcs = np.array(rotation_mvr)
C_mvr_wrt_lcs = np.transpose(np.array([traslation_mvr]))
P_mvr_wrt_lcs = utils.create_pose(R_mvr_wrt_lcs, C_mvr_wrt_lcs)

print(traslation_mvr)
print(quaternion_mvr)

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)
plot_cs(P_rv_wrt_lcs, 'rv', L=1)
plot_cs(P_mvl_wrt_lcs, 'mvl', L=1)
plot_cs(P_mvr_wrt_lcs, 'mvr', L=1)

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')